In [1]:
from seleniumwire import webdriver
import time
import pathlib
import os
from urllib.parse import urlparse, parse_qs

# get driver path
current_path = pathlib.Path().resolve()
driver_path = os.path.join(current_path, "chromedriver")

driver = webdriver.Chrome(driver_path)

driver.get('https://data.eco-counter.com/public2/?&id=100019726')

# wait for page to load competletely
time.sleep(3)

In [2]:
# get all requests made by the page
all_requests = driver.requests

In [3]:
TARGET_URL = "www.eco-visio.net"
GET_REQUEST = ""

Filter out the get request made to the database

In [4]:
for request in all_requests:
    # parse url and filter out query param
    parsed_url = urlparse(request.url)
    query_dictionary = parse_qs(parsed_url.query)
    # check if target url matches and has a token in query
    if (parsed_url.netloc == TARGET_URL and "t" in query_dictionary):
        GET_REQUEST = request
        print(GET_REQUEST.url)
        break

https://www.eco-visio.net/api/aladdin/1.0.0/pbl/publicwebpage/data/100019726?begin=20150601&end=20210921&step=4&domain=4701&withNull=true&t=4a26c833613dbe87908c38a9136cb2e1a9dbe76af315f5458ec121345d775831


In [5]:
query_dictionary = parse_qs(parsed_url.query)
query_dictionary

{'begin': ['20150601'],
 'end': ['20210921'],
 'step': ['4'],
 'domain': ['4701'],
 'withNull': ['true'],
 't': ['4a26c833613dbe87908c38a9136cb2e1a9dbe76af315f5458ec121345d775831']}